In [1]:
import os
os.environ["HF_HOME"] = "/cs/student/projects3/aisd/2024/ghanda/hf-cache"
os.environ["TRANSFORMERS_CACHE"] = "/cs/student/projects3/aisd/2024/ghanda/hf-cache"
os.environ["HF_HUB_CACHE"] = "/cs/student/projects3/aisd/2024/ghanda/hf-cache"

In [2]:
!pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 47.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 53.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 53.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 MB 26.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 45.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 35.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 56.3 MB/s et

In [17]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from huggingface_hub import login

login(token="REMOVED***REMOVED***")

df = pd.read_csv("personality_data_train.csv")
dataset = Dataset.from_pandas(df)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
# Add a pad token if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

def tokenize(example):
    input_text = example["Question"]
    target_text = example["Answer"]
    model_inputs = tokenizer(
        input_text,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=256
        )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

/cs/student/msc/aisd/2024/ghanda/venvs/uclenv/lib64/python3.9/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
# ✅ STEP 5: LOAD MODEL WITH QLoRA
import torch
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

model.resize_token_embeddings(len(tokenizer))  # Resize for new pad token if added

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   8%|8         | 917M/10.9G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:  17%|#7        | 961M/5.50G [00:00<?, ?B/s]

OSError: [Errno 122] Disk quota exceeded